In [ ]:
import geopandas as gpd
import pandas as pd
from glob import glob
import numpy as np
import math
import gc
import psutil

In [ ]:
import os

# Define the folder path you want to check （定义你想要检查的文件夹路径）
folder_path = '/mnt/ramdisk' 

# Check if the folder exists （检查文件夹是否存在）
if not os.path.exists(folder_path):
    # If the folder does not exist, create the folder （如果文件夹不存在，则创建文件夹）
    os.makedirs(folder_path)

from pandarallel import pandarallel
os.environ['JOBLIB_TEMP_FOLDER'] = '/mnt/ramdisk' 
pandarallel.initialize(nb_workers=6,progress_bar=True,verbose=2,use_memory_fs = False ) 

In [ ]:
def delete_file(folder_path):
    # Use the glob module to find all files in a folder （使用 glob 模块找到文件夹中的所有文件）
    files = glob(folder_path + '/*')
    # Iterate through the file list and delete each file using os.remove （遍历文件列表，使用 os.remove 删除每个文件）
    for file in files:
        os.remove(file)

In [ ]:
def now_ram():
    mem_info = psutil.Process(os.getpid()).memory_info().rss
    print('python ram usage：%.4f GB' % (mem_info / 1024 / 1024 / 1024))

In [ ]:
def function(x):
    name="..."
    #... Add the appropriate content ... A dataframe usually contains a geometry attribute, which is stored as a pickle.
    #(... 添加相应内容 ... dataframe 中一般是包含 geometry 属性的，先用pickle的形式暂存)
    x.to_pickle("/mnt/ramdisk/%s.pkl"%name)

In [ ]:
def check_new_pid(seen_pids, new_pids=set(), first=False):
    for proc in psutil.process_iter(['pid', 'cmdline']):
        # Check command line parameters (检查命令行参数)
        if '/usr/bin/python3' in proc.info['cmdline']:
            #print(proc.info['cmdline'])
            child_processes = proc.children(recursive=True)
            #print(child_processes)
            if first==True:
                for child in child_processes:
                    seen_pids.add(child.pid)
                return seen_pids
            else:
                for child in child_processes:
                    if child.pid not in seen_pids:
                        new_pids.add(child.pid)
                return new_pids

In [ ]:
# Check existing python processes (检查现有的python进程)
seen_pids = check_new_pid(seen_pids=set(), first=True)
print(seen_pids)

In [ ]:
delete_file("/mnt/ramdisk")
os.system("mount -t tmpfs -o size=10000M tmpfs /mnt/ramdisk")
print("Mounting the virtual disk is completed")
print("挂载虚拟盘完成")

AS=gpd.read_file("./your_shp_path")
AS.parallel_apply(lambda x: function(x),axis=1)
print("Distribution calculation complete")
print("分布计算完成")

now_ram()

df_list=[pd.read_pickle(i) for i in glob("/mnt/ramdisk/*.pkl")] # Must be saved to a variable, otherwise the memory will not be recycled (一定要保存到一个变量中，否则内存不回收)

now_ram()

buffer_AS=pd.concat(df_list)

del df_list # Be sure to reclaim memory immediately (一定要立即回收内存)
gc.collect()

now_ram()
print("Merge complete")
print("合并完成")

#... Add the appropriate content ... Generally, the operations on the merged buffer_AS
#... 添加相应内容 ... 一般是对合并后的 buffer_AS 进行相关的操作

output.to_file("./your_output_shp_path") # Export shp (导出shp)
print("Export complete")
print("导出完成")

now_ram()

del buffer_AS #Clean up used variables to prevent memory from not being reclaimed (把用过的变量清理掉，防止内存不回收)
gc.collect()

now_ram()

# Check for emerging python processes and kill the ones that don't exit automatically.
# (检查新出现的python进程，kill掉没有自动退出的子进程)
new_pid=check_new_pid(seen_pids=seen_pids)
for pid in new_pid:
    # Use the kill -9 command to stop the process (使用 kill -9 命令停止进程)
    os.system(f'kill -9 {pid}')

now_ram()

# Clean up the cache files (对缓存文件进行清理)
delete_file("/mnt/ramdisk")
os.system('umount -l /mnt/ramdisk')
delete_file("/dev/shm")
Print("Cleanup complete")
print("清理完成")